# Dewan Lab Neuron Pseudopopulation Analysis
## Import Dependencies

In [ ]:
import itertools
import os
os.environ['ISX'] = '0'

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from tqdm.auto import tqdm

from dewan_calcium import classifiers, plotting
from dewan_calcium import stats as dewan_stats
from dewan_calcium.helpers import IO, trace_tools
from dewan_calcium.helpers.project_folder import ProjectFolder

pd.options.mode.copy_on_write = "warn"

print('Finished importing required libraries!')

## Load Data from Project Folder

In [ ]:
# Create Project Folder to Gather and Hold all the File Paths
project_folder = ProjectFolder('ODOR', combined=True)

In [ ]:
# If this is the first time the project folder has been created,
# move the files to the appropriate directories and then run this cell, otherwise skip this cell
project_folder.get_data()

## Configs


In [ ]:
ANALYSIS_VARIABLE = 'odor' # Either 'odor', 'class' or 'block'

CELL_CLASS = 'vglut' # vglut or vgat

MAX_NUM_CELLS = 0
ENTROPY = None

WINDOW = 2 # Window size for the moving-window SVM decoder; set to None for no window and to consider all data at once
NUM_SVM_SPLITS = 20  # Number of random test-train splits to run and average per SVM run

# Values used in the combine-data.py standalone script to define the sizes of the different data periods
BASELINE_FRAMES = 20
ODOR_FRAMES = 30
POST_FRAMES = 10

SHOW_FIGURES = False

## Constants

In [ ]:
## ============================ CONSTANTS ============================ ##
VARS = ['odor', 'class', 'block']
CLASSES = ['vglut', 'vgat', 'oxtr']

CONC_ORDER = {
    '5ATE1': 'ATE', '5ATE10': 'ATE', '5ATE100' : 'ATE', '5ATE1000': 'ATE',
    '5AL1': 'AL', '5AL10' : 'AL', '5AL100' : 'AL', '5AL1000' : 'AL',
    '5AMINE1' : 'AMINE', '5AMINE10' : 'AMINE', '5AMINE100' : 'AMINE', '5AMINE1000' : 'AMINE',
    '5OL1' : 'OL', '5OL10' : 'OL', '5OL100' : 'OL', '5OL1000' : 'OL',
    '5ONE1' : 'ONE', '5ONE10' : 'ONE', '5ONE100' : 'ONE', '5ONE1000' : 'ONE'
}

SEPARATE_CONC_ORDER = {

    'AL': ['5AL1', '5AL10', '5AL100', '5AL1000'],
    'ATE': ['5ATE1', '5ATE10', '5ATE100', '5ATE1000'],
    'AMINE': ['5AMINE1', '5AMINE10', '5AMINE100', '5AMINE1000'],
    'OL': ['5OL1', '5OL10', '5OL100', '5OL1000'],
    'ONE': ['5ONE1', '5ONE10', '5ONE100', '5ONE1000']
}

ODOR_BASE = ['ATE', 'AL', 'AMINE', 'OL', 'ONE']

LEVEL_MAP = {
    'class': (1,2,3), # Level = 3 if we want to only look at class; drops odors and blocks
    'odor': (2,3), # Level = 2 if we want to only look at odors; drops block and class labels
    'block': (1,3,4) # level = 1 if we want to only look at blocks; drops odor and class labels
}

CM_WINDOWS = {
   'Baseline': (0,20),
   'Odor_Period': (20,40),
   'Latent_Period': (40,60),
   'Odor_and_Latent': (20,60)
}

## VALIDATE INPUTS

IO.verify_input('ANALYSIS_VARIABLE', ANALYSIS_VARIABLE, [str], allowed_values=VARS)
IO.verify_input('CELL_CLASS', CELL_CLASS, [str], allowed_values=CLASSES)
IO.verify_input('WINDOW', WINDOW, [int], allowed_range=(1, 20))
IO.verify_input('NUM_SVM_SPLITS', NUM_SVM_SPLITS, [int], allowed_range=(1, 100), inclusive=True)

## Select Labels

if ANALYSIS_VARIABLE == 'odor':
        _exp_type = 'CONC'
        _labels=list(CONC_ORDER.keys())
        _classes = CONC_ORDER
        _separate_labels = SEPARATE_CONC_ORDER
elif ANALYSIS_VARIABLE == 'class':
        _exp_type = 'Conc-Class'
        _labels=ODOR_BASE
else: # Blocks
    _exp_type = 'Conc-Blocks'
    _labels=[1, 2, 3]


data_file = []
sig_table = []

if project_folder.raw_data_dir.combined_data_path:
    if CELL_CLASS.lower() in str(project_folder.raw_data_dir.combined_data_path).lower():
        data_file = project_folder.raw_data_dir.combined_data_path[0]
    else:
        raise FileExistsError(f'No data file with class {CELL_CLASS} exists!')

if project_folder.raw_data_dir.combined_sig_table_path:
    if CELL_CLASS.lower() in str(project_folder.raw_data_dir.combined_sig_table_path).lower():
        sig_table = project_folder.raw_data_dir.combined_sig_table_path[0]
    else:
        raise FileExistsError(f'No significance table with class {CELL_CLASS} exists!')

unfiltered_data_file = data_file.parent.joinpath(f'{CELL_CLASS}_Comb_unfiltered-combined.pkl')


## Load and Normalize Data

In [ ]:
# If desired, N number of cells can be dropped so one dataset can match another in number of cells.
# Set MAX_NUM_CELLS to the upper limit, and X cells (num_cells - MAX_NUM_CELLS) will be chosen at random to be dropped from our current dataset
# If desired, the entropy value used to construct the rng will be printed, and can be saved/reused to have the exact same cells dropped on each run
seed_sequence = np.random.SeedSequence(entropy=ENTROPY)
print(f'Last Entropy Value was: {seed_sequence.entropy}')
rng = np.random.default_rng(seed_sequence)

combined_data = pd.read_pickle(data_file, compression={'method': 'xz'})
combined_data_unfiltered = pd.read_pickle(unfiltered_data_file, compression={'method': 'xz'})
if MAX_NUM_CELLS > 0:
    cells = combined_data.columns.get_level_values(0).unique()
    num_cells = len(cells)
    num_cells_to_drop = num_cells - MAX_NUM_CELLS
    if num_cells_to_drop > 0:
        random_cells = rng.choice(cells, num_cells_to_drop, replace=False)
        combined_data = combined_data.T.drop(random_cells).T
    else:
        raise ValueError(f'{MAX_NUM_CELLS} is too large! You cannot drop all cells, or more cells than exist in the dataset')

## Add the class for each odor to the column
cells = np.unique(combined_data.columns.get_level_values(0).values)
combined_data, original_columns = classifiers.add_odor_class(combined_data, _classes)

In [ ]:
# Run this cell to reset the dataframe to its original configuration
combined_data.columns = original_columns

## SVM Classifier

### Sliding Window Decoding Directories

In [ ]:
## Load/Create output directories

svm_output_dir = project_folder.analysis_dir.output_dir.subdir('SVM')
svm_fig_dir = project_folder.analysis_dir.figures_dir.subdir('SVM')

svm_output_dir = svm_output_dir.joinpath(ANALYSIS_VARIABLE)
svm_fig_dir = svm_fig_dir.joinpath(ANALYSIS_VARIABLE)
svm_output_dir.mkdir(parents=True, exist_ok=True)
svm_fig_dir.mkdir(parents=True, exist_ok=True)

cm_data_save_dir = svm_output_dir.joinpath('CM')
cm_figure_save_dir = svm_fig_dir.joinpath('CM')
cm_data_save_dir.mkdir(parents=True, exist_ok=True)
cm_figure_save_dir.mkdir(parents=True, exist_ok=True)

#### While similar to 'class' mode, this will determine the classifiers ability to descriminate between each version of an odor instead of CLASS A v. CLASS B v. CLASS C, etc.


In [ ]:
num_cells = len(cells)

grouped_by_odor = combined_data.T.groupby('Class') # Group by class so each odor is separated

odor_segmented_svm_results = {odor: {} for odor in ODOR_BASE}
odor_segmented_shuffled_svm_results = {odor: {} for odor in ODOR_BASE}


if WINDOW:
    svm_output_dir = svm_output_dir.joinpath(f'Window-{WINDOW}')
    svm_fig_dir = svm_fig_dir.joinpath(f'Window-{WINDOW}')
    svm_output_dir.mkdir(exist_ok=True, parents=True)
    svm_fig_dir.mkdir(exist_ok=True, parents=True)

for odor, odor_df in grouped_by_odor:

    odor_output_dir = svm_output_dir.joinpath(odor)
    odor_svm_fig_dir = svm_fig_dir.joinpath(odor)
    odor_CM_data_dir = cm_data_save_dir.joinpath(odor)
    odor_CM_fig_dir = cm_figure_save_dir.joinpath(odor)
    odor_output_dir.mkdir(parents=True, exist_ok=True)
    odor_svm_fig_dir.mkdir(parents=True, exist_ok=True)
    odor_CM_data_dir.mkdir(parents=True, exist_ok=True)
    odor_CM_fig_dir.mkdir(parents=True, exist_ok=True)

    odor_df = odor_df.T
    odor_df.columns = odor_df.columns.droplevel(2) # No longer need class names, so drop them
    labels = _separate_labels[odor]

    mean_svm_scores, splits_v_repeat_df, all_confusion_mats, (true_labels, pred_labels) = classifiers.sliding_window_ensemble_decoding(odor_df, window_size=WINDOW, num_splits=NUM_SVM_SPLITS, class_labels=labels)
    mean_score_df = classifiers.postprocess(mean_svm_scores, num_cells, window=WINDOW)
    IO.save_SVM_output(odor_output_dir, mean_score_df, mean_svm_scores, splits_v_repeat_df, all_confusion_mats, true_labels, pred_labels)

    shuffled_data = classifiers.shuffle_data(odor_df)
    shuffled_mean_svm_scores, shuffled_splits_v_repeat_df, shuffled_all_confusion_mats, (shuffled_true_labels, shuffled_pred_labels) = classifiers.sliding_window_ensemble_decoding(shuffled_data, window_size=WINDOW, num_splits=NUM_SVM_SPLITS, class_labels=labels)
    shuffled_mean_score_df = classifiers.postprocess(shuffled_mean_svm_scores, num_cells, window=WINDOW)
    IO.save_SVM_output(odor_output_dir, shuffled_mean_score_df, shuffled_mean_svm_scores, shuffled_splits_v_repeat_df, shuffled_all_confusion_mats, shuffled_true_labels, shuffled_pred_labels, True)

    # Preprocess SVM Data
    mean_performance, CI_min, CI_max = classifiers.preprocess_for_plotting(mean_svm_scores, splits_v_repeat_df)
    shuffle_mean_performance, shuffle_CI_min, shuffle_CI_max = classifiers.preprocess_for_plotting(shuffled_mean_svm_scores, shuffled_splits_v_repeat_df)
    CI = (CI_min, CI_max)
    shuffle_CI = (shuffle_CI_min, shuffle_CI_max)

    # Save SVM Performance Data
    _index = [(int(item[0]), int(item[1])) for item in list(mean_svm_scores.keys())]
    classifiers.save_svm_data(mean_performance, shuffle_mean_performance, _index, CI, shuffle_CI, odor_output_dir)

    odor_segmented_svm_results[odor] = {
        'mean_svm_scores': mean_svm_scores,
        'splits_v_repeat_df': splits_v_repeat_df,
        'all_confusion_mats': all_confusion_mats,
        'true_labels': true_labels,
        'pred_labels': pred_labels,
    }

    odor_segmented_shuffled_svm_results[odor] = {
        'mean_svm_scores': shuffled_mean_svm_scores,
        'splits_v_repeat_df': shuffled_splits_v_repeat_df,
        'all_confusion_mats': shuffled_all_confusion_mats,
        'true_labels': shuffled_true_labels,
        'pred_labels': shuffled_pred_labels,
    }


    # Plot individual odor segmentated SVM Performance
    descriptors = (_exp_type, CELL_CLASS, ANALYSIS_VARIABLE, len(cells))
    svm_fig = plotting.plot_svm_performance(mean_performance, shuffle_mean_performance, CI, shuffle_CI, descriptors, odor_svm_fig_dir)

    if SHOW_FIGURES:
        plt.show()
    else:
        plt.close()

    windows = list(all_confusion_mats.keys())
    window_averaged_cms = classifiers.average_CM(all_confusion_mats, windows)

    # Plot and Save average confusion matrices per window
    for window_name in CM_WINDOWS:
        cm_window = CM_WINDOWS[window_name]

        _fig = classifiers.save_and_plot_CM(window_averaged_cms, cm_window, window_name, windows, labels, odor_CM_data_dir, odor_CM_fig_dir)

        if not SHOW_FIGURES:
            plt.close()

In [ ]:
svm_means = []
shuffled_svm_means = []
all_vals = []
shuffled_vals = []

for odor in odor_segmented_svm_results:
    mean_scores = odor_segmented_svm_results[odor]['mean_svm_scores']
    shuffled_mean_scores = odor_segmented_shuffled_svm_results[odor]['mean_svm_scores']
    means = [mean_scores[key] for key in mean_svm_scores]
    shuffled_means = [shuffled_mean_scores[key] for key in shuffled_mean_scores]
    svm_means.append(means)
    shuffled_svm_means.append(shuffled_means)

    splits_v_repeat_df = odor_segmented_svm_results[odor]['splits_v_repeat_df']
    shuffled_splits_v_repeat_df = odor_segmented_shuffled_svm_results[odor]['splits_v_repeat_df']
    all_vals.append(splits_v_repeat_df)
    shuffled_vals.append(shuffled_splits_v_repeat_df)


In [ ]:

average_svm_performance = np.mean(svm_means, axis=0)
average_shuffled_svm_performance = np.mean(shuffled_svm_means, axis=0)
average_svm_splits = pd.DataFrame(np.mean(all_vals, axis=0))
average_shuffled_splits = pd.DataFrame(np.mean(shuffled_vals, axis=0))

average_svm_performance_dict = {}
average_shuffled_svm_performance_dict = {}
for i in range(len(average_svm_performance)):
    average_svm_performance_dict[i] = average_svm_performance[i]
    average_shuffled_svm_performance_dict[i] = average_shuffled_svm_performance[i]

mean_performance, CI_min, CI_max = classifiers.preprocess_for_plotting(average_svm_performance_dict, average_svm_splits)
shuffle_mean_performance, shuffle_CI_min, shuffle_CI_max = classifiers.preprocess_for_plotting(average_shuffled_svm_performance_dict, average_shuffled_splits)

CI = (CI_min, CI_max)
shuffle_CI = (shuffle_CI_min, shuffle_CI_max)

# Save SVM Performance Data
_index = [(int(item[0]), int(item[1])) for item in list(mean_svm_scores.keys())]
classifiers.save_svm_data(mean_performance, shuffle_mean_performance, _index, CI, shuffle_CI, svm_output_dir)

## Cell Number SVM Decoding

In [ ]:
run_order = []
# Shuffle the order of cells
num_cells = np.arange(len(cells))
shuffled_cells = cells.copy()
rng.shuffle(shuffled_cells)

# Create a list of lists that progressively adds cells
# E.g. [c1], [c1, c2], [c1, c2, c3], ...
for i in num_cells:
    run_order.append(shuffled_cells[num_cells[:(i + 1)]])

classifier_data = combined_data.iloc[BASELINE_FRAMES: BASELINE_FRAMES + ODOR_FRAMES, :]
classifier_data.columns = classifier_data.columns.droplevel(LEVEL_MAP[ANALYSIS_VARIABLE])
grouped_by_class = classifier_data.T.groupby('Class') # Group by class so each odor is separated

for class_name, class_data in grouped_by_class:
    results = []

    mean_scores = {}
    splits = {}
    confusion_mats = {}
    true_labels = {}
    pred_labels = {}

    print(f'Running Neuron Number SVM Decoder for {class_name}')
    cell_number_svm_stats = pd.DataFrame(index=num_cells+1, columns=['mean', 'ci_min', 'ci_max'])
    odor_output_dir = project_folder.analysis_dir.output_dir.subdir('SVM').joinpath('cell_number').joinpath(class_name)
    odor_output_dir.mkdir(parents=True, exist_ok=True)
    class_data.index = class_data.index.droplevel(level=2) # Dont need classes anymore
    labels = _separate_labels[class_name]

    classifier_results = classifiers.pooled_neuron_decoding(class_data.T, run_order, labels)

    # Parse Output
    for i, result in enumerate(tqdm(classifier_results)):
        cell_num = i + 1 # i is zero-indexed
        mean_svm_scores, splits_v_repeat_df, all_confusion_mats, (_true, _pred) = result

        # Each 'run' is made up of N repeats we should average together
        mean_performances, CI_mins, CI_maxs = classifiers.preprocess_for_plotting(mean_svm_scores, splits_v_repeat_df)
        CI_min = np.mean(CI_mins)
        CI_max = np.mean(CI_maxs)
        mean_performance = np.mean(mean_performances)
        cell_number_svm_stats.loc[cell_num] = [mean_performance, CI_min, CI_max]

        mean_scores[cell_num] = mean_svm_scores
        splits[cell_num] = splits_v_repeat_df
        confusion_mats[cell_num] = all_confusion_mats
        true_labels[cell_num] = _true
        pred_labels[cell_num] = _pred

    IO.save_SVM_output(odor_output_dir, cell_number_svm_stats, mean_scores, splits, confusion_mats, true_labels, pred_labels)

In [ ]:

# Plot SVM Performance
descriptors = (_exp_type, CELL_CLASS, ANALYSIS_VARIABLE, len(cells))

svm_fig = plotting.plot_svm_performance(mean_performance, shuffle_mean_performance, CI, shuffle_CI, descriptors, svm_fig_dir)

if not SHOW_FIGURES:
    plt.close()

windows = list(all_confusion_mats.keys())
window_averaged_cms = classifiers.average_CM(all_confusion_mats, windows)

# Plot and Save average confusion matrices per window
for window_name in CM_WINDOWS:
    cm_window = CM_WINDOWS[window_name]

    _fig = classifiers.save_and_plot_CM(window_averaged_cms, cm_window, window_name, windows, ['1', '10', '100', '1000'], cm_data_save_dir, cm_figure_save_dir)

    if not SHOW_FIGURES:
        plt.close()


## Population and Lifetime Sparseness


### Calculate Cell-Odor Means


In [ ]:
if ANALYSIS_VARIABLE != 'odor':
    raise ValueError('Population and Lifetime sparseness can only be run if analyzing odorants!')

# Pop Sparseness -> per odor
# Lifetime Sparseness -> per cell

combined_data.columns = original_columns
transposed_data = combined_data.T

combined_data_unfiltered.columns = combined_data_unfiltered.columns.rename('Odor', level=1)
transposed_data_unfiltered = combined_data_unfiltered.T

cell_means, cell_medians, nonzero_cell_means, nonzero_cell_medians = trace_tools.dff_magnitude_means(transposed_data, _labels, BASELINE_FRAMES, ODOR_FRAMES, subset=0)

# For our magnitude graph, we need a copy of the data with every cell--including nonresponsive cells; we also want to include MO and Buzzer in their dataset
_labels_with_mo_buzzer = _labels.copy()
_labels_with_mo_buzzer.extend(['MO', 'Buzzer'])
unfiltered_cell_means, unfiltered_cell_medians, unfiltered_nonzero_cell_means, unfiltered_nonzero_cell_medians = trace_tools.dff_magnitude_means(transposed_data_unfiltered, _labels_with_mo_buzzer, BASELINE_FRAMES, ODOR_FRAMES, subset=0)

In [ ]:
# Plot dF/F Magnitude Figure
cell_names = nonzero_cell_means.columns.get_level_values(0).unique()
output_dir = project_folder.analysis_dir.figures_dir.subdir('DFF')
_ = plotting.plot_all_avg_dff(unfiltered_nonzero_cell_means.T, output_dir)

## Population Sparseness


In [ ]:
if ANALYSIS_VARIABLE != 'odor':
    raise ValueError('Population and Lifetime sparseness can only be run if analyzing odorants!')

pop_sparseness_values_means = {}
pop_sparseness_values_medians = {}

for odor_name, odor_data in cell_means.iterrows():
    num_cells = odor_data.shape[0]
    odor_data = odor_data.values
    sparseness_value = dewan_stats.sparseness(num_cells, odor_data)
    pop_sparseness_values_means[odor_name] = sparseness_value

for odor_name, odor_data in cell_medians.iterrows():
    num_cells = odor_data.shape[0]
    odor_data = odor_data.values
    sparseness_value = dewan_stats.sparseness(num_cells, odor_data)
    pop_sparseness_values_medians[odor_name] = sparseness_value

pop_sparseness_values_means = pd.DataFrame(pop_sparseness_values_means, index=[0])
pop_sparseness_values_medians = pd.DataFrame(pop_sparseness_values_medians, index=[0])

pop_sparseness_values_means = pop_sparseness_values_means.fillna('X')
pop_sparseness_values_medians = pop_sparseness_values_medians.fillna('X')

## Lifetime Sparseness

In [ ]:
if ANALYSIS_VARIABLE != 'odor':
    raise ValueError('Population and Lifetime sparseness can only be run if analyzing odorants!')

lifetime_sparseness_values_means = {}
lifetime_sparseness_values_medians = {}
print('Lifetime Sparseness (Means):')
for cell_name, cell_data in cell_means.items():
    num_odors = cell_data.shape[0]
    cell_data = cell_data.values
    sparseness_values = dewan_stats.sparseness(num_odors, cell_data)
    if sparseness_values is np.nan:
        print(f'Cell {cell_name} returned NaN')
    lifetime_sparseness_values_means[cell_name] = sparseness_values

print('Lifetime Sparseness (Medians):')
for cell_name, cell_data in cell_medians.items():
    num_odors = cell_data.shape[0]
    cell_data = cell_data.values
    sparseness_values = dewan_stats.sparseness(num_odors, cell_data)
    if sparseness_values is np.nan:
        print(f'Cell {cell_name} returned NaN')
    lifetime_sparseness_values_medians[cell_name] = sparseness_values

lifetime_sparseness_values_means = pd.DataFrame(lifetime_sparseness_values_means, index=[0])
lifetime_sparseness_values_medians = pd.DataFrame(lifetime_sparseness_values_medians, index=[0])

lifetime_sparseness_values_means = lifetime_sparseness_values_means.fillna('X')
lifetime_sparseness_values_medians = lifetime_sparseness_values_medians.fillna('X')

## Write to File


In [ ]:
if ANALYSIS_VARIABLE != 'odor':
    raise ValueError('Population and Lifetime sparseness can only be run if analyzing odorants!')

sparseness_path = project_folder.analysis_dir.output_dir.path.joinpath('sparseness.xlsx')

with pd.ExcelWriter(sparseness_path, engine='xlsxwriter') as writer:
    pop_sparseness_values_medians.to_excel(writer, sheet_name='Population Sparseness (Medians)')
    pop_sparseness_values_means.to_excel(writer, sheet_name='Population Sparseness (Means)')
    lifetime_sparseness_values_medians.to_excel(writer, sheet_name='Lifetime Sparseness (Medians)', index=[0])
    lifetime_sparseness_values_means.to_excel(writer, sheet_name='Lifetime Sparseness (Means)', index=[0])
    cell_means.to_excel(writer, sheet_name='Cell Means (Zeroed)')
    cell_medians.to_excel(writer, sheet_name='Cell Medians (Zeroed)')

## Correlations

In [ ]:
if ANALYSIS_VARIABLE != 'odor':
    raise ValueError('Correlations can only be run if analyzing odorants!')


odors = transposed_data.index.get_level_values(1).unique()
odors = np.sort(odors)
perms = list(itertools.permutations(odors, 2))
correlations = pd.DataFrame(dtype=float, index=odors, columns=odors)  # Explicitly set dtype to float

for odor1, odor2 in perms:
    odor1_means = cell_means.loc[odor1]
    odor2_means = cell_means.loc[odor2]

    pearson_result = stats.pearsonr(odor1_means, odor2_means)

    correlations.loc[odor1, odor2] = pearson_result.statistic

correlations = correlations.fillna(1.0)
correlations = 1 - correlations

sorted_correlations = correlations.loc[_labels, _labels]

correlation_path = project_folder.analysis_dir.output_dir.path.joinpath('correlations.xlsx')
sorted_correlations.to_excel(correlation_path)

## Reorganize Cell Significance Matrix


## Sorting Rules
1) Non-responsive - zeros only
2) Excitatory responses - 1 only (sort by the most 1s)
3) Inhibitory responses - -1 only (sort by the most -1s)
4) Combo responses (any combination of 1/-1 sorted by the most responses)
5) Buzzer - any cell that responses to the buzzer (sort by the total number of responses of any number)
6) MO - any cell that responses to MO (sort by the total number of responses of any number)


In [ ]:
def sort_by_response_number(df: pd.DataFrame):
    order = df.T.ne(0).sum().sort_values(ascending=False).index
    sorted_df = df.loc[order]
    return sorted_df

def get_and_sort_cells(odor_responsive_cells, IDs):
    ID_mask = np.all(odor_responsive_cells.isin(IDs), axis=1)
    ID_cells = odor_responsive_cells.loc[ID_mask]
    sorted_ID_cells = sort_by_response_number(ID_cells)
    return sorted_ID_cells

combined_sig_table = pd.read_excel(sig_table, index_col=0)

responsive_cells_mask = (combined_sig_table != 0).sum(axis=1) > 0
responsive_cells = combined_sig_table.loc[responsive_cells_mask]

nonresponsive_cells_mask = np.logical_not(responsive_cells_mask)
nonresponsive_cells = combined_sig_table.loc[nonresponsive_cells_mask]

buzzer_mask = (responsive_cells['Buzzer'] != 0)
buzzer_cells = responsive_cells.loc[buzzer_mask]
buzzer_cells = sort_by_response_number(buzzer_cells)

non_buzzer_mask = np.logical_not(buzzer_mask)
non_buzzer_cells = responsive_cells.loc[non_buzzer_mask]

MO_mask = (non_buzzer_cells['MO'] != 0)
MO_cells = non_buzzer_cells.loc[MO_mask]
MO_cells = sort_by_response_number(MO_cells)

non_MO_mask = np.logical_not(MO_mask)
odor_responsive_cells = non_buzzer_cells.loc[non_MO_mask]

excitatory_on_cells = get_and_sort_cells(odor_responsive_cells, [0, 1])   # 0 and 1 ONLY
odor_responsive_cells = odor_responsive_cells.drop(excitatory_on_cells.index)

inhibitory_on_cells = get_and_sort_cells(odor_responsive_cells, [0, -1])  # 0 and 1 ONLY
odor_responsive_cells = odor_responsive_cells.drop(inhibitory_on_cells.index)

any_combo_cells = sort_by_response_number(odor_responsive_cells)

sorted_dataframe = pd.concat([nonresponsive_cells, excitatory_on_cells, inhibitory_on_cells, any_combo_cells, buzzer_cells, MO_cells])


_order = np.hstack([_labels, ['Buzzer', 'MO']])
sorted_by_odor = sorted_dataframe[_order]

sorted_sig_table_path = project_folder.analysis_dir.output_dir
IO.save_sorted_significance_table(sorted_by_odor, sorted_sig_table_path)